# Call the fortran routine to get the drift velocity

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use({'figure.dpi':100})

Define a routine that calculates the locations of the grid interfaces.

In [ ]:
def get_interfaces_from_log_cell_centers(x):
    """
    Returns the cell interfaces for an array of logarithmic
    cell centers.
    
    Arguments:
    ----------
    
    x : array
    :   Array of logarithmically spaced cell centers for
        which the interfaces should be calculated
        
    Output:
    -------
    
    xi : array
    :    Array of length len(x)+1 containing the grid interfaces
         defined such that 0.5*(xi[i]+xi[i+1]) = xi
    """
    x = np.asarray(x)
    B = x[1]/x[0]
    A = (B+1)/2.
    xi = np.append(x/A,x[-1]*B/A)
    return xi

Import the fortran routine.

In [ ]:
try:
    from calc_drift import routines
except ImportError:
    print('Will try to import directly compiled routines ...')
    import sys
    sys.path.append('calc_drift')
    import routines
fortran_routine = routines.duststep_donorcell

Get some constants from the fortran module. 

In [ ]:
au    = routines.constants.au
rho_s = routines.variables.rho_s

You can also set some of the module values such as

```python

routines.variables.m_star = 2*routines.constants.m_sun # for a 2 solar mass star
routines.switches.dust_diffusion = 1

```

in case use used different settings.

# Call the routine

Initialize the quantities - *use your own input here*

In [ ]:
n_r                = 200
g2d                = 100.
x                  =  np.logspace(-1,3,n_r)*au
x05                = get_interfaces_from_log_cell_centers(x)[:-1]
grainsize          = 1e0
grainmass          = 4*np.pi/3.*rho_s*grainsize**3
sigma_g            = 100*(x/au)**-1*(x<100*au)+1e-100
sigma_1            = np.outer(grainsize**(0.5),x)
sigma_d            = sigma_g/g2d
T                  = 200*(x/au)**-1
sigma_dot          = np.zeros_like(x)
v_gas              = np.zeros_like(x)
alpha              = np.ones_like(x)
coagulation_method = 1

Make a simplified time loop

In [ ]:
n_t  = 100
year = 3.15e7
time = np.logspace(0,5,n_t)*year
tc   = 0.
out  = []
sigma1 = sigma_d.copy()

for t in time:
    #
    # calculate time step and update current time
    #
    dt  = t-tc
    tc += dt
    #
    # call the routine and store all output for every time step in the list `out`
    #
    _ = fortran_routine(dt, grainsize, grainmass, x, x05, sigma_g, T,\
                        sigma_dot, sigma1, v_gas, alpha, coagulation_method, n_r)
    out += [_]
    #
    # assign the output to reasonable variable names and update the curent value
    #
    sigma2, dust_accretion, dust_flux_o, v_drift, v_05, flim, diff, dust_flux, a, b, c, d = _
    sigma1 = sigma2.copy()

# Plotting

### Plot surface density change

In [ ]:
f,ax = plt.subplots()
for i in [0,-1]:
    ax.loglog(x/au,out[i][0],label='time = {:.2g} year'.format(time[i]/year))
ax.set_xlabel('r [au]')
ax.set_ylabel('$\Sigma_\mathrm{dust}$ [g cm$^{-2}$]')
ax.set_ylim(1e-5,1e5)
ax.legend(fontsize='small',loc='best');

### Plot drift velocities

In [ ]:
f,ax = plt.subplots()
i = 0
ax.loglog(x/au,-out[i][3],label='v_drift'.format(time[i]/year))
ax.loglog(x05/au,-out[i][4],label='v_05'.format(time[i]/year))
ax.set_xlabel('r [au]')
ax.set_ylabel('$-v_\mathrm{r}$ [cm s$^{-1}$]')
ax.set_title('negative dust velocity')
ax.set_ylim(1e-5,1e5)
ax.legend(fontsize='small',loc='best');